In [1]:
import sklearn as sk
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import csv
import sklearn.feature_extraction.text as sk_text
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
import random

KeyboardInterrupt: 

In [ ]:
# Make Reviews tsv file with business_id, stars, and text
outfile=open('review_stars.tsv', 'w')
sfile=csv.writer(outfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','stars','text']) #column titles

with open('yelp_dataset/yelp_academic_dataset_review.json', encoding="utf-8") as f:
    for line in f:
        row=json.loads(line)
        sfile.writerow([row['business_id'],row['stars'],(row['text']).encode('utf-8')])
outfile.close()

In [ ]:
# Make business tsv file
outfile=open('business_stars.tsv', 'w')
sfile=csv.writer(outfile,delimiter='\t',quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','categories', 'name'])

with open('yelp_dataset/yelp_academic_dataset_business.json', encoding='utf-8') as f:
    for line in f:
        row = json.loads(line)
        if(row['review_count']>30):
            sfile.writerow([row['business_id'], row['categories'], (row['name']).encode('utf-8')])
outfile.close()

In [ ]:
df_reviews=pd.read_csv('review_stars.tsv', sep='\t')
df_business=pd.read_csv('business_stars.tsv', sep='\t')
df_merged=pd.merge(df_reviews, df_business, on='business_id')
df_merged.to_csv('merged_df.tsv',sep='\t')
del df_reviews
del df_business
del df_merged

# Data Processing

In [ ]:
df_merged=pd.read_csv('merged_df.tsv', sep='\t')

In [ ]:
# Limiting number of reviews due to size
numEntries = 100000

In [ ]:
vectorizer = sk_text.TfidfVectorizer(
                                    max_features=600,
                                    max_df=2000,
                                    min_df=1)
matrix=vectorizer.fit_transform(df_merged.text[0:numEntries])
tfidf_data=matrix.toarray()
del matrix

x = tfidf_data
y = df_merged.stars[0:numEntries]

## Finding Best Options

## Variation attempts
Layers:[20,20,20,20,1], activation:relu, RMSE: 0.9809  
Layers:[100,50,25,10,1], activation:relu, RMSE: 0.9976  
Layers:[20,20,1], activation:relu, RMSE: 0.9948  
Layers:[20,20,20,20,1], activation:tanh, RMSE: 0.9805  
Layers:[20,20,20,20,20,20,20,20,1], activation:tanh, RMSE: 0.9789 <- Best  
Layers:[20,20,20,20,20,20,20,20,1], activation:sigmoid, RMSE: 0.9890  
Layers:[20,20,20,1], activation:sigmoid, RMSE: 0.9879  
Layers:[100,50,20,1], activation:sigmoid, RMSE: 0.9879  
Layers:[20,20,20,20,20,20,20,20,1], activation:tanh, optimzer: sgd, RMSE: 0.9900


In [ ]:
model = Sequential()

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=32)

model.add(Dense(20, input_dim=x.shape[1], activation='tanh')) 
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(20, activation='tanh'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=100)

model.load_weights('dnn/best_weights.hdf5') # load weights from best model

In [ ]:
pred=model.predict(x_test)

In [ ]:
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final RMSE score: {:.4}".format(score))

## Plotting regression

In [ ]:
# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.to_numpy()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

In [ ]:
chart_regression(pred.flatten()[:200],y_test[:200],sort=True)

# Individual Business Predictions

In [ ]:
df_unique_ids = df_merged.drop_duplicates(subset=['business_id']).reset_index()

In [ ]:
def predict_single_business(id_num):
    df_single_business = df_merged[df_merged.business_id == id_num]
    vectorizer = sk_text.TfidfVectorizer(
                                    max_features=600,
                                    max_df=2000,
                                    min_df=1)
    matrix=vectorizer.fit_transform(df_single_business.text)
    tfidf_data=matrix.toarray()
    if(tfidf_data.shape[1] >= 500):
        return df_single_business.name[:1].to_string(index=False)[1:].strip("'").strip('"'), model.predict(tfidf_data).mean(), df_single_business.stars.mean()

In [ ]:
def single_business_compare(numBusinesses):
    for i in range(numBusinesses):
        index = random.randint(0, df_unique_ids.shape[0])
        name, predicted, real = predict_single_business(df_unique_ids.business_id[index])
        print("{:30}: predicted score: {:3.1f}, real score: {:3.1f}".format(name, predicted, real))

In [ ]:
single_business_compare(5)